# Imports

## System imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import json
import re

import seaborn as sns
import os

## Data imports

v1: predbezne vysledky po errate, ulozene na SharePointe OSN. Este sa budu updatovat

In [2]:
all_inpatients_file_path = r"C:\Users\hunakj\health.gov.sk\IZA - Cestovanie za ZS\2023\Data inputs\osn_vsetka_starostlivost_2023_ms_v2024.2.2.csv"

In [3]:
all_inpatients = pd.read_csv(all_inpatients_file_path, dtype=str, delimiter=";")

In [4]:
all_inpatients.head()

,id_hp,vek_roky,vek_dni,hmotnost,upv,diagnozy,vykony,drg,erv,typ_starostlivosti,diagnozy_z_01,vykony_z_01,obdobie_od,obdobie_do,datum_od,datum_do,osetrovacia_doba,id_poistenca,datum_narodenia,kod_pobytu,kod_zp,pzs_12,IDENTIFZAR,pzs_8,pzs_6,id_hp_pzs,ms
0,35380_23000079,20,NaN,NaN,NaN,j350,5g603&B&20230113,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-12,2023-01-16,4,5130405,2002-04-25,522279,24,N61173014101,N6117301,N6117301,N61173,23000079,S05-34~S05-29
1,35380_23000214,19,NaN,NaN,NaN,j350,5g603&B&20230127,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-26,2023-01-31,5,2778465,2003-08-23,527696,24,N61173014101,N6117301,N6117301,N61173,23000214,S05-34~S05-29
2,35380_23000126,19,NaN,NaN,NaN,j350~j352,5g603&B&20230118~5g6071&Z&20230118,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-17,2023-01-21,4,3305786,2003-10-07,529249,24,N61173014101,N6117301,N6117301,N61173,23000126,S05-34~S05-29
3,35380_23000083,18,NaN,NaN,NaN,j350,5g603&B&20230113,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-12,2023-01-17,5,3305359,2004-08-15,520128,24,N61173014101,N6117301,N6117301,N61173,23000083,S53-33~S63-40
4,35380_23000159,18,NaN,NaN,NaN,h912,NaN,d61a,0.7048,DRG,h912,NaN,2023-01,2023-01,2023-01-20,2023-01-26,6,2852825,2004-08-31,520802,24,N61173014101,N6117301,N6117301,N61173,23000159,S53-29


In [5]:
all_inpatients.dtypes

id_hp                 object
vek_roky              object
vek_dni               object
hmotnost              object
upv                   object
diagnozy              object
vykony                object
drg                   object
erv                   object
typ_starostlivosti    object
diagnozy_z_01         object
vykony_z_01           object
obdobie_od            object
obdobie_do            object
datum_od              object
datum_do              object
osetrovacia_doba      object
id_poistenca          object
datum_narodenia       object
kod_pobytu            object
kod_zp                object
pzs_12                object
IDENTIFZAR            object
pzs_8                 object
pzs_6                 object
id_hp_pzs             object
ms                    object
dtype: object

In [6]:
all_inpatients.shape

(1178065, 27)

## Hospitals information

In [7]:
pzs_gps = pd.read_excel(r"C:\Users\hunakj\health.gov.sk\IZA - Cestovanie za ZS\2023\Data inputs\other\PZS_zakl_info_GPS_bezprepojeni.xlsx", dtype=str)

In [8]:
pzs_gps.rename(columns={'IDENTIFZAR': 'pzs_8', 'SIDZAR_KOD_OBEC': 'KOD_OBEC_PZS', 'SIDZAR_KOD_OKRES_PZS': 'KOD_OKRES_PZS', 'lat_pzs_ifexists': 'lat_pzs', 'lng_pzs_ifexists': 'lng_pzs', 'NAZZAR': 'NAZOV_PZS', 'OSN_siet' : 'OSN_siet', 'OSN_level' : 'OSN_level', 'typ_nemocnice' : 'typ_nemocnice'}, inplace=True)

In [9]:
pzs_gps.head(2)

,pzs_8,PZS_short,NAZOV_PZS,OSN_siet,OSN_level,typ_nemocnice,KOD_OBEC_PZS,KOD_OKRES_PZS,lat_pzs,lng_pzs,NAZZAR_povodne,lat_pzs_ifexists_povodne,lng_pzs_ifexists.1,SIDZAR_KOD_OBEC_povodne,SIDZAR_KOD_OKRES_PZS_povodne
0,P5848001,P58480,"Ministerstvo vnútra Slovenskej republiky, všeo...",nie,mimo siet OSN,mimo siet OSN,501760,201,47.8001114823,17.6645174625,"Ministerstvo vnútra Slovenskej republiky, všeo...",47.8001114823,17.6645174625,501760,201
1,P5848002,P58480,"Ministerstvo vnútra Slovenskej republiky, všeo...",nie,mimo siet OSN,mimo siet OSN,520004,702,48.9336706585,21.9065813542,"Ministerstvo vnútra Slovenskej republiky, všeo...",48.9336706585,21.9065813542,520004,702


## Patient information

In [10]:
pacient_gps = pd.read_excel(r"C:\Users\hunakj\health.gov.sk\IZA - Cestovanie za ZS\2023\Data inputs\other\obce_gps_simple.xlsx")

In [11]:
pacient_gps = pacient_gps.rename(columns={'RESIDENCY': 'kod_pobytu'})

In [12]:
pacient_gps.head()

,kod_pobytu,Obec_pacient_nazov,id_okres_pacient,Okres_pacient_nazov,okresne_mesto_zuj_pacient,okresne_mesto_pacient_nazov,lat_okres_pacient,lng_okres_pacient,residency_count
0,501280,Nesvady,401,Komárno,501026,Komárno,47.765799,18.119551,1
1,504025,Sala,405,Šaľa,504025,Šaľa,48.156715,17.875971,1
2,504092,Trnovec nad Vahom,405,Šaľa,504025,Šaľa,48.156715,17.875971,1
3,555509,Zalesie,108,Senec,508217,Senec,48.219813,17.396061,1
4,504131,Velke Ulany,202,Galanta,503665,Galanta,48.189953,17.725027,1


In [13]:
pacient_gps.dtypes

kod_pobytu                      object
Obec_pacient_nazov              object
id_okres_pacient                object
Okres_pacient_nazov             object
okresne_mesto_zuj_pacient       object
okresne_mesto_pacient_nazov     object
lat_okres_pacient              float64
lng_okres_pacient              float64
residency_count                  int64
dtype: object

## MS names and levels

Tento zoznam treba "oholit" na uplne jednoduchu tabulku bez duplicit, inak to robi problemy a duplikuje riadky. To znamena, ze treba dat prec aj zdielane sluzby. Upravoval som este samotny excel.

In [14]:
ms_names_levels = pd.read_excel(r"C:\Users\hunakj\health.gov.sk\IZA - Cestovanie za ZS\2023\Data inputs\other\02_Zoznam-medicinskych-sluzieb 1 adjusted.xlsx")

In [15]:
ms_names_levels.head()

,Číslo programu,Kód medicínskej služby,ms_code,Zdieľaná medicínska služba,ms_name,typ,level_dospely,level_do1r,level_1az6,level_7az15,level_16az18
0,1,S01-04,S01-04,NaN,Bezvedomie a zabezpečenie DC vyžadujúce úzko š...,V,IV,NaN,NaN,NaN,NaN
1,1,S01-03,S01-03,NaN,Podpora dýchania vyžadujúca úzko špecializovan...,V,IV,NaN,NaN,NaN,NaN
2,1,S01-02,S01-02,NaN,Podpora krvného obehu vyžadujúca úzko špeciali...,V,IV,NaN,NaN,NaN,NaN
3,1,S01-01,S01-01,NaN,Kritické poruchy vnútorného prostredia vyžaduj...,V,IV,NaN,NaN,NaN,NaN
4,1,S01-14,S01-14,NaN,Komplexná intenzívna starostlivosť po transpla...,D,III,NaN,NaN,NaN,NaN


## List of medical programmes

In [16]:
medical_programmes_list = pd.read_excel(r"C:\Users\hunakj\health.gov.sk\IZA - Cestovanie za ZS\2023\Data inputs\other\Medicinske_programy_zoznam.xlsx")

In [17]:
medical_programmes_list.head()

,Oznacenie_prislusnych_MS_SXX,Cislo_programu,Nazov_programu,Cislo_a_nazov_programu
0,S01,1,Program anestéziológie a intenzívnej medicíny,1 Program anestéziológie a intenzívnej medicíny
1,S02,2,Traumatologický program,2 Traumatologický program
2,S03,3,Neurochirurgický program,3 Neurochirurgický program
3,S04,4,Oftalmologický program,4 Oftalmologický program
4,S05,5,Otorinolaryngologický program,5 Otorinolaryngologický program


# Data processing

## Choose first MS 

In [18]:
all_inpatients['ms_code'] = all_inpatients['ms'].str.slice(0,6)

In [19]:
all_inpatients.head()

,id_hp,vek_roky,vek_dni,hmotnost,upv,diagnozy,vykony,drg,erv,typ_starostlivosti,diagnozy_z_01,vykony_z_01,obdobie_od,obdobie_do,datum_od,datum_do,osetrovacia_doba,id_poistenca,datum_narodenia,kod_pobytu,kod_zp,pzs_12,IDENTIFZAR,pzs_8,pzs_6,id_hp_pzs,ms,ms_code
0,35380_23000079,20,NaN,NaN,NaN,j350,5g603&B&20230113,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-12,2023-01-16,4,5130405,2002-04-25,522279,24,N61173014101,N6117301,N6117301,N61173,23000079,S05-34~S05-29,S05-34
1,35380_23000214,19,NaN,NaN,NaN,j350,5g603&B&20230127,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-26,2023-01-31,5,2778465,2003-08-23,527696,24,N61173014101,N6117301,N6117301,N61173,23000214,S05-34~S05-29,S05-34
2,35380_23000126,19,NaN,NaN,NaN,j350~j352,5g603&B&20230118~5g6071&Z&20230118,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-17,2023-01-21,4,3305786,2003-10-07,529249,24,N61173014101,N6117301,N6117301,N61173,23000126,S05-34~S05-29,S05-34
3,35380_23000083,18,NaN,NaN,NaN,j350,5g603&B&20230113,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-12,2023-01-17,5,3305359,2004-08-15,520128,24,N61173014101,N6117301,N6117301,N61173,23000083,S53-33~S63-40,S53-33
4,35380_23000159,18,NaN,NaN,NaN,h912,NaN,d61a,0.7048,DRG,h912,NaN,2023-01,2023-01,2023-01-20,2023-01-26,6,2852825,2004-08-31,520802,24,N61173014101,N6117301,N6117301,N61173,23000159,S53-29,S53-29



## Merge MS names

In [20]:
ms_names_levels_NamesOnly = ms_names_levels[['ms_code', 'ms_name']]

In [21]:
ms_names_levels_NamesOnly.head()

,ms_code,ms_name
0,S01-04,Bezvedomie a zabezpečenie DC vyžadujúce úzko š...
1,S01-03,Podpora dýchania vyžadujúca úzko špecializovan...
2,S01-02,Podpora krvného obehu vyžadujúca úzko špeciali...
3,S01-01,Kritické poruchy vnútorného prostredia vyžaduj...
4,S01-14,Komplexná intenzívna starostlivosť po transpla...


In [22]:
all_inpatients = pd.merge(all_inpatients, ms_names_levels_NamesOnly, on='ms_code', how='left')

## Based on age, assign ms level

In [23]:
ms_names_levels_AllLevels = ms_names_levels[['ms_code', 'level_dospely', 'level_do1r', 'level_1az6', 'level_7az15', 'level_16az18']]

In [24]:
ms_names_levels_AllLevels.head()

,ms_code,level_dospely,level_do1r,level_1az6,level_7az15,level_16az18
0,S01-04,IV,NaN,NaN,NaN,NaN
1,S01-03,IV,NaN,NaN,NaN,NaN
2,S01-02,IV,NaN,NaN,NaN,NaN
3,S01-01,IV,NaN,NaN,NaN,NaN
4,S01-14,III,NaN,NaN,NaN,NaN


In [25]:
all_inpatients = pd.merge(all_inpatients, ms_names_levels_AllLevels, on='ms_code', how='left')

In [26]:
def determine_ms_level(row):
    if row['vek_roky'] <= 1:
        return row['level_do1r']
    elif row['vek_roky'] <= 6:
        return row['level_1az6']
    elif row['vek_roky'] <= 15:
        return row['level_7az15']
    elif row['vek_roky'] <= 18:
        return row['level_16az18']    
    else:
        return np.nan

In [27]:
nan_count_vek_roky = all_inpatients['vek_roky'].isna().sum()
print(nan_count_vek_roky)

2


kedze su len 2 NaN values, dam ich prec, inak by bolo treba napisat funkciu, aby sa NaNs dali napr. na minus 1

In [28]:
all_inpatients = all_inpatients.dropna(subset=['vek_roky'])

In [29]:
all_inpatients['vek_roky'] = all_inpatients['vek_roky'].astype(int)

In [30]:
all_inpatients['ms_level'] = all_inpatients.apply(determine_ms_level, axis=1)

In [31]:
all_inpatients['ms_level'] = all_inpatients['ms_level'].fillna(all_inpatients['level_dospely'])

In [32]:
nan_count_ms_level = all_inpatients['ms_level'].isna().sum()
print(nan_count_ms_level)

103


In [33]:
filtered_df = all_inpatients.loc[all_inpatients['id_hp'] == "24283_23005692"]

In [34]:
filtered_df

,id_hp,vek_roky,vek_dni,hmotnost,upv,diagnozy,vykony,drg,erv,typ_starostlivosti,diagnozy_z_01,vykony_z_01,obdobie_od,obdobie_do,datum_od,datum_do,osetrovacia_doba,id_poistenca,datum_narodenia,kod_pobytu,kod_zp,pzs_12,IDENTIFZAR,pzs_8,pzs_6,id_hp_pzs,ms,ms_code,ms_name,level_dospely,level_do1r,level_1az6,level_7az15,level_16az18,ms_level
1620,24283_23005692,17,NaN,NaN,NaN,r55,321030&Z&20230221~34017&Z&20230223~8u001&Z&202...,f73z,0.6039,DRG,r55,NaN,2023-02,2023-02,2023-02-20,2023-02-23,3,8773524,2005-02-24,524603,24,N33067007101,N3306701,N3306701,N33067,23005692,S68-24~S64-20,S68-24,MR v oblasti brucha a panvy,NaN,III,III,NaN,NaN,NaN


zostava mi stale okolo 100 NaNs, mozno chyba vo vykazovani, lebo take pripady nemaju level vo svojej kategorii podla veku a ani pre dospeleho. 

In [35]:
#later delete 
all_inpatients = all_inpatients.drop(columns=['level_dospely', 'level_do1r', 'level_1az6', 'level_7az15', 'level_16az18'])

In [36]:
all_inpatients.shape

(1178063, 30)

## Adding type of MS

In [37]:
ms_names_levels_msTypes = ms_names_levels[['ms_code', 'typ']]

In [38]:
all_inpatients = pd.merge(all_inpatients, ms_names_levels_msTypes, on='ms_code', how='left')

In [39]:
all_inpatients.shape

(1178063, 31)

## Adding PZS (hospital) GPS and additional information

In [40]:
all_inpatients = all_inpatients.merge(pzs_gps[['pzs_8','NAZOV_PZS', 'OSN_siet', 'OSN_level', 'typ_nemocnice', 'KOD_OKRES_PZS', 'lat_pzs', 'lng_pzs']], on='pzs_8', how='left')

## Adding patient GPS and disctrict

In [41]:
all_inpatients['kod_pobytu'] = all_inpatients['kod_pobytu'].astype(str)
pacient_gps['kod_pobytu'] = pacient_gps['kod_pobytu'].astype(str)

In [42]:
all_inpatients = all_inpatients.merge(pacient_gps[['kod_pobytu','Obec_pacient_nazov', 'id_okres_pacient', 'Okres_pacient_nazov', 'lat_okres_pacient', 'lng_okres_pacient']], on='kod_pobytu', how='left')

In [43]:
all_inpatients['id_okres_pacient'].isna().sum()

np.int64(1769)

## Add main programme number and name

In [44]:
#adding main programme name
all_inpatients = pd.merge(all_inpatients,
    medical_programmes_list,
    left_on=all_inpatients['ms_code'].str[:3],
    right_on=medical_programmes_list['Oznacenie_prislusnych_MS_SXX'].str[:3],
    how='left'
    )

In [45]:
#delete unnecessary columns after adding main programme name
all_inpatients = all_inpatients.drop(columns=['key_0', 'Oznacenie_prislusnych_MS_SXX', 'Cislo_programu', 'Nazov_programu'])

## Preliminary output

In [46]:
all_inpatients.shape

(1178063, 44)

In [47]:
all_inpatients['id_hp'].nunique()

1178063


v 2022 som mal menej unikatnych 'id_hp' ako celkovo riadkov, cize boli asi nejake duplicity. Tu nie su, co je super. 

In [48]:
all_inpatients.head()

,id_hp,vek_roky,vek_dni,hmotnost,upv,diagnozy,vykony,drg,erv,typ_starostlivosti,diagnozy_z_01,vykony_z_01,obdobie_od,obdobie_do,datum_od,datum_do,osetrovacia_doba,id_poistenca,datum_narodenia,kod_pobytu,kod_zp,pzs_12,IDENTIFZAR,pzs_8,pzs_6,id_hp_pzs,ms,ms_code,ms_name,ms_level,typ,NAZOV_PZS,OSN_siet,OSN_level,typ_nemocnice,KOD_OKRES_PZS,lat_pzs,lng_pzs,Obec_pacient_nazov,id_okres_pacient,Okres_pacient_nazov,lat_okres_pacient,lng_okres_pacient,Cislo_a_nazov_programu
0,35380_23000079,20,NaN,NaN,NaN,j350,5g603&B&20230113,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-12,2023-01-16,4,5130405,2002-04-25,522279,24,N61173014101,N6117301,N6117301,N61173,23000079,S05-34~S05-29,S05-34,Tonzilektómia alebo adenoidektómia,I,V,"ORL HUMENNÉ, s.r.o., špecializovaná nemocnica",ano,3,Špecializovaná nemocnica,702,48.934469,21.898742,Michalovce,807,Michalovce,48.754228,21.910748,5 Otorinolaryngologický program
1,35380_23000214,19,NaN,NaN,NaN,j350,5g603&B&20230127,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-26,2023-01-31,5,2778465,2003-08-23,527696,24,N61173014101,N6117301,N6117301,N61173,23000214,S05-34~S05-29,S05-34,Tonzilektómia alebo adenoidektómia,I,V,"ORL HUMENNÉ, s.r.o., špecializovaná nemocnica",ano,3,Špecializovaná nemocnica,702,48.934469,21.898742,Okruhle,712,Svidník,49.306993,21.569144,5 Otorinolaryngologický program
2,35380_23000126,19,NaN,NaN,NaN,j350~j352,5g603&B&20230118~5g6071&Z&20230118,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-17,2023-01-21,4,3305786,2003-10-07,529249,24,N61173014101,N6117301,N6117301,N61173,23000126,S05-34~S05-29,S05-34,Tonzilektómia alebo adenoidektómia,I,V,"ORL HUMENNÉ, s.r.o., špecializovaná nemocnica",ano,3,Špecializovaná nemocnica,702,48.934469,21.898742,Vysna Sitnica,702,Humenné,48.933671,21.906581,5 Otorinolaryngologický program
3,35380_23000083,18,NaN,NaN,NaN,j350,5g603&B&20230113,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-12,2023-01-17,5,3305359,2004-08-15,520128,24,N61173014101,N6117301,N6117301,N61173,23000083,S53-33~S63-40,S53-33,Tonzilektómia alebo adenoidektómia,I,V,"ORL HUMENNÉ, s.r.o., špecializovaná nemocnica",ano,3,Špecializovaná nemocnica,702,48.934469,21.898742,Certizne,705,Medzilaborce,49.275750,21.905336,53 Otorinolaryngologický program pre deti
4,35380_23000159,18,NaN,NaN,NaN,h912,NaN,d61a,0.7048,DRG,h912,NaN,2023-01,2023-01,2023-01-20,2023-01-26,6,2852825,2004-08-31,520802,24,N61173014101,N6117301,N6117301,N61173,23000159,S53-29,S53-29,"Poruchy rovnováhy, strata sluchu alebo tinnitus",III,D,"ORL HUMENNÉ, s.r.o., špecializovaná nemocnica",ano,3,Špecializovaná nemocnica,702,48.934469,21.898742,Snina,709,Snina,48.987886,22.158990,53 Otorinolaryngologický program pre deti


# Outputs

## All inpatients aggregated by hospital, patient district and medical service

In [49]:
all_inpatients_pivot = all_inpatients.groupby(['pzs_8', 'NAZOV_PZS', 'id_okres_pacient', 'ms_level', 'ms_code', 'ms_name']).agg({
    'Okres_pacient_nazov': 'first',  # Assuming the name doesn't change within groups
    'OSN_siet' : 'first',
    'OSN_level' : 'first',
    'typ_nemocnice' : 'first',
    'lat_okres_pacient': 'first',  # Assuming each hospital has a unique set of coordinates
    'lng_okres_pacient': 'first',
    'id_hp': 'size'  # If 'patient_id' uniquely identifies patients
}).reset_index().rename(columns={'id_hp': 'patient_count_By_hospital_PatientDistrict_MSlevels'})

In [50]:
all_inpatients_pivot = all_inpatients_pivot.sort_values(by=[ 'patient_count_By_hospital_PatientDistrict_MSlevels'], ascending=[ False])

In [51]:
all_inpatients_pivot['ms_code_name'] = all_inpatients_pivot['ms_code'] + " " + all_inpatients_pivot['ms_name']

In [52]:
# Priradujem nazov hlavneho programu
all_inpatients_pivot = pd.merge(all_inpatients_pivot,
    medical_programmes_list,
    left_on=all_inpatients_pivot['ms_code'].str[:3],
    right_on=medical_programmes_list['Oznacenie_prislusnych_MS_SXX'].str[:3],
    how='left'
    )



In [53]:
# Odstranujem nepotrebne stlpce po priradeni hlavneho programu
all_inpatients_pivot = all_inpatients_pivot.drop(columns=['key_0', 'Oznacenie_prislusnych_MS_SXX', 'Cislo_programu', 'Nazov_programu'])

In [54]:
all_inpatients_pivot.shape

(145794, 15)

In [55]:
all_inpatients_pivot.head()

,pzs_8,NAZOV_PZS,id_okres_pacient,ms_level,ms_code,ms_name,Okres_pacient_nazov,OSN_siet,OSN_level,typ_nemocnice,lat_okres_pacient,lng_okres_pacient,patient_count_By_hospital_PatientDistrict_MSlevels,ms_code_name,Cislo_a_nazov_programu
0,P4070701,Univerzitná nemocnica Bratislava,101,I,S04-43,Terapeutické podanie látok do oka,Bratislava I-V,ano,5,Všeobecná nemocnica,48.147992,17.100952,3396,S04-43 Terapeutické podanie látok do oka,4 Oftalmologický program
1,P4070703,Univerzitná nemocnica Bratislava,101,I,S04-43,Terapeutické podanie látok do oka,Bratislava I-V,ano,5,Všeobecná nemocnica,48.147992,17.100952,3348,S04-43 Terapeutické podanie látok do oka,4 Oftalmologický program
2,P8109501,"Fakultná nemocnica s poliklinikou Nové Zámky, ...",404,I,S04-43,Terapeutické podanie látok do oka,Nové Zámky,ano,2,Všeobecná nemocnica,47.989298,18.170745,2867,S04-43 Terapeutické podanie látok do oka,4 Oftalmologický program
3,P8568701,"Fakultná nemocnica Nitra, všeobecná nemocnica",403,I,S04-43,Terapeutické podanie látok do oka,Nitra,ano,3,Všeobecná nemocnica,48.309149,18.079528,2264,S04-43 Terapeutické podanie látok do oka,4 Oftalmologický program
4,N4223101,Fakultná nemocnica s poliklinikou F.D.Roosevel...,601,I,S04-43,Terapeutické podanie látok do oka,Banská Bystrica,ano,4,Všeobecná nemocnica,48.730425,19.138477,2208,S04-43 Terapeutické podanie látok do oka,4 Oftalmologický program


In [56]:
all_inpatients_pivot.to_csv("all_inpatients_pivot.csv")

## Hospitals, coordinates and patient counts

In [57]:
hospital_coordinates_patientcounts = all_inpatients.groupby(['pzs_8', 'NAZOV_PZS',]).agg({
    'lat_pzs': 'first',  # Assuming each hospital has a unique set of coordinates
    'lng_pzs': 'first',
    'OSN_siet' : 'first',
    'OSN_level' : 'first',
    'typ_nemocnice' : 'first',
    'id_hp': 'size'  # If 'patient_id' uniquely identifies patients
}).reset_index().rename(columns={'id_hp': 'patient_count_hospital'})

In [58]:
hospital_coordinates_patientcounts.shape

(310, 8)

In [59]:
hospital_coordinates_patientcounts.head()

,pzs_8,NAZOV_PZS,lat_pzs,lng_pzs,OSN_siet,OSN_level,typ_nemocnice,patient_count_hospital
0,N1855722,"Spišská katolícka charita, Hospic Sv. Alžbety,...",49.1218172755,20.446369775,nie,mimo siet OSN,mimo siet OSN,102
1,N1949101,"Sanatórium Tatranská Kotlina, n.o., špecializo...",49.1445802074,20.2442966301,nie,mimo siet OSN,mimo siet OSN,991
2,N1968101,Národný ústav detskej tuberkulózy a respiračný...,49.132872,20.238762,ano,5,Špecializovaná nemocnica,2433
3,N1969701,"Sanatórium Dr. Guhra, n.o., špecializovaná nem...",49.1445802074,20.2442966301,nie,mimo siet OSN,mimo siet OSN,632
4,N2114901,"Kysucká nemocnica s poliklinikou Čadca, všeobe...",49.439411,18.772795,ano,2,Všeobecná nemocnica,13108


In [60]:
#hospital_coordinates_patientcounts.to_csv("hospital_coordinates_patientcounts.csv")